# Plot structure and PCA components

What type of information does our PCA decomposition in vertex-space actually capture?

In [3]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
sys.path.append('../src')
from utils import plot_face

In [4]:
N = 848  # number of unique stimuli
n_v = 31049  # number of vertices

# Load vertices, both static and dynamic
v = np.load('../data/vertices.npz')['v']
vs = v[:, 0, :, :]
vd = v[:, 1, :, :]

mean_face = vs.mean(axis=0)  # for plotting
tris = np.load('../data/tris.npy') - 1  # triangles

FileNotFoundError: [Errno 2] No such file or directory: '../data/vertices.npz'

## Dynamic face movements
Importantly, we operationalized the movement of vertices as difference between the vertex coordinates at frame 1 and the vertex coordinates at frame 15 (the frame containing the peak expression). Then, we fit a PCA on this difference.

We redo this here, to create a figure that shows the (cumulative) explained variance.

In [ ]:
pca_vs = PCA(n_components=50)
pca_vs.fit(vs.reshape((N, -1)))
pca_vd = PCA(n_components=50)
pca_vd.fit(vd.reshape((N, -1)))

In [ ]:
fig, axes = plt.subplots(nrows=2, sharex=True, sharey=True, figsize=(10, 8))
axes[0].bar(range(1, 51), pca_vd.explained_variance_ratio_)
axes[0].set_ylabel('Explained variance ratio', fontsize=15)

ax_tw = axes[0].twinx()
c_evr = np.r_[0, np.cumsum(pca_vd.explained_variance_ratio_)]
ax_tw.plot(range(51), c_evr, c='k', ls='--')
ax_tw.set_ylim(0, 1)
ax_tw.set_ylabel("Cumulative expl. variance ratio", fontsize=15)

axes[1].bar(range(1, 51), pca_vs.explained_variance_ratio_)
axes[1].set_ylabel('Explained variance ratio', fontsize=15)

ax_tw = axes[1].twinx()
c_evr = np.r_[0, np.cumsum(pca_vs.explained_variance_ratio_)]
ax_tw.plot(range(0, 51), c_evr, c='k', ls='--')
ax_tw.set_ylim(0, 1.003)
ax_tw.set_ylabel("Cumulative expl. variance ratio", fontsize=15)

axes[0].set_xlim(-.1, 50)
axes[0].set_ylim(0, 0.45)
axes[0].text(25, 0.225, "Dynamic", va='center', ha='center', fontsize=25)
axes[1].text(25, 0.225, "Static", va='center', ha='center', fontsize=25)
axes[1].set_xlabel("Component nr.", fontsize=15)

sns.despine(top=True, right=False)
fig.tight_layout(h_pad=3)
fig.savefig('../figures/pca_decomposition.png', dpi=200)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(15, 8), sharex=True, sharey=True)

for i, tpe in enumerate(['dynamic', 'static']):
    coef = pd.read_csv(f'../results/validation/target-emotion_fs-vertexPCA_type-{tpe}_coefs.tsv', sep='\t', index_col=0)
    for emo in ['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise']:
        tmp = coef.query("emotion == @emo").drop(['emotion', 'feature_set', 'sub', 'icept'], axis=1)
        tmp = tmp.mean(axis=0).abs().to_numpy()
        tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
        axes[0, i].plot(tmp)
        if i == 0:
            axes[0, i].legend(['Anger', 'Disgust', 'Fear', 'Happiness', 'Sadness', 'Surprise'],
                              frameon=False, fontsize=15)


    for target in ['valence', 'arousal']:
        coef = pd.read_csv(f'../results/validation/target-{target}_fs-vertexPCA_type-{tpe}_coefs.tsv', sep='\t', index_col=0)
        tmp = coef.drop(['icept', 'sub', 'feature_set', 'sigma'], axis=1)
        tmp = tmp.mean(axis=0).abs().to_numpy()
        tmp = (tmp - tmp.min()) / (tmp.max() - tmp.min())
        axes[1, i].plot(tmp)
        if i == 0:
            axes[1, i].legend(['Valence', 'Arousal'], frameon=False, fontsize=15)
        
for i in range(2):
    axes[i, 0].set_ylabel(r'$\mathrm{abs}(\hat{\beta})$', fontsize=20)
    axes[1, i].set_xlabel('Component', fontsize=20)
sns.despine()
fig.tight_layout()

First, we'll load the PCA parameters ("mu", the mean, and "w", the weights) of this decomposition:

In [ ]:
with np.load('../results/pca/pca_type-dynamic_weights.npz') as data:
    # "vd" stands for "vertex difference"
    mu_vd, W_vd = data['mu'], data['W']
    
# Note that the 3 coordinates (X, Y, Z) per vertex are flattened here
print(mu_vd.shape)
print(W_vd.shape)  # 50 PCA components

Because (almost) each stimulus contained *some* movement, the `mu` variable contains the average movement, which we ignore for now. 

We can also plot separate components to see which vertices are "grouped" into a component. To do so, we define a face in PCA space (which has 50 dimensions) with a particular configuration. For example, if we want to visualize a face with only PCA component 1, we could specify its configuration as:

\begin{align}
X_{\mathrm{comp} = 1} = [1, 0, 0, ..., 0]
\end{align}

However, setting it to "1" is rather arbitrary in PCA space. It makes more sense to set it to, let's say, 3 standard deviations of that feature, i.e.:

\begin{align}
X_{\mathrm{comp} = 1} = [3\times \hat{\sigma}_{X_{1}}, 0, 0, ..., 0]
\end{align}

In [ ]:
# First PC
vd_pca = pd.read_csv('../data/features/vertexPCA_type-dynamic.tsv', sep='\t', index_col=0)
vd_pca_std = vd_pca.std(axis=0).to_numpy()

IDX = 2
X = np.zeros((1, 50))
X[0, IDX] = vd_pca_std[IDX] * 3
print(X)

Then, we invert the PCA transformation (i.e., the "inverse PCA") to go from PCA space ($X$) to stimulus space ($S$):

\begin{align}
S = XW + \mu
\end{align}

where $W$ are the PCA weights and $\mu$ is the PCA mean.

The visualization has three faces, highlighting the movement in X (left-right), Y (up-down), and Z (front-back) separately. Blue = more to the left (X), down (Y), and back (Z). Red = more to right (X), up (Y), and front (Z).

In [ ]:
S = (X @ W_vd + mu_vd).reshape((n_v, 3))
overlay = S / vd.std(axis=0)
fig = plot_face(mean_face + S, tris, overlay=overlay)
fig.show()

Let's do it for the first four components:

In [ ]:
S = np.zeros((4, n_v, 3))
for idx in range(4):
    X = np.zeros((1, 50))
    X[0, idx] = vd_pca_std[IDX] * 3
    S[idx, :, :] = (X @ W_vd + mu_vd).reshape((n_v, 3))
    
overlay = S / vd.std(axis=0)
fig = plot_face(mean_face + S, tris, overlay=overlay, cmin=-2, cmax=2)
fig.write_image('../figures/type-dynamic_pca.png', scale=2)
fig.show() 

## Static face information

In [ ]:
with np.load('../results/pca/pca_type-static_weights.npz') as data:
    mu_vs, W_vs = data['mu'], data['W']

Let's plot the average face.

In [ ]:
fig = plot_face(mu_vs.reshape((n_v, 3)), tris)
fig.show()

And let's plot the first PCA component. Note that we leave out the mean of the PCA ($\mu$) in the inverse transformation.

In [ ]:
vs_pca = pd.read_csv('../data/features/vertexPCA_type-static.tsv', sep='\t', index_col=0)
vs_pca_std = vs_pca.std(axis=0).to_numpy()

S = np.zeros((4, n_v, 3))
for idx in range(4):
    X = np.zeros((1, 50))
    X[0, idx] = vs_pca_std[IDX] * 3
    S[idx, :, :] = (X @ W_vs).reshape((n_v, 3))
    
overlay = S / vs.std(axis=0)
fig = plot_face(mean_face + S, tris, overlay=overlay, cmin=-3, cmax=3)
fig.write_image('../figures/type-static_pca.png', scale=2)
fig.show() 